In [ ]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import time as t
import datetime
import os
import shutil
from zipfile import ZipFile

In [ ]:
path = '/content/drive/MyDrive/Degree Project/dataset/csv_format/'
dataset_path = '/content/drive/MyDrive/Degree Project/dataset/'
extracts = '/content/drive/MyDrive/Degree Project/dataset/extracts/'
local_path = '/content/'
calendar_df = pd.read_csv(path + 'calendar_dates.csv')

In [ ]:
route_data = pd.read_csv(extracts + 'route_data.csv')
trip_data = pd.read_csv(extracts + 'trip_data.csv')

routes = list(route_data.route_id.unique())

FMT = '%H:%M:%S'

In [ ]:
def get_delay(end_time, start_time):
  x = t.strptime(end_time, FMT)
  y = t.strptime(start_time, FMT)
  xminutes = (datetime.timedelta(hours=x.tm_hour,minutes=x.tm_min,seconds=x.tm_sec).total_seconds())/60
  yminutes = (datetime.timedelta(hours=y.tm_hour,minutes=y.tm_min,seconds=y.tm_sec).total_seconds())/60
  return y - x

In [ ]:
# End of shifts for a given route
class EOS:
  def __init__(self):
    self.eos = {}
  
  def add_eos(self, route, eoss):
    if not type(eoss) == list:
      raise Exception('expected a list for eos')
    else:
      self.eos[route] = eoss
  
  def update_eos(self, route, new_eoss):
    if not type(new_eoss) == list:
      raise Exception('expected a list for eos')
    else:
      self.eos[route] = new_eoss
  
  def get_eos(self):
    return self.eos

In [ ]:
class trip_set:
  def __init__(self):
      self.trip_sets = {}

  def export_trip_sets(self):
    trip_set_path = extracts + 'trip_sets'
    if not os.path.exists(trip_set_path):
      os.mkdir(trip_set_path)
    for key in list(self.trip_sets.keys()):
      route_trips = pd.DataFrame.from_dict(self.trip_sets[key], orient='index').transpose()
      route_trips.to_csv('{}.csv'.format(key), index = False)
      shutil.move(local_path + '{}.csv'.format(key), trip_set_path + '/{}.csv'.format(key))

  def create_trip_set(self, route, index):
    TS_route_index = 'TS_{}_{}'.format(route, index)
    self.trip_sets[TS_route_index] = {}
    self.trip_sets[TS_route_index]['route_id'] = []
    self.trip_sets[TS_route_index]['trip_id'] = []
    self.trip_sets[TS_route_index]['trip_headsign'] = []
    self.trip_sets[TS_route_index]['origin'] = []
    self.trip_sets[TS_route_index]['destination'] = []
    self.trip_sets[TS_route_index]['start_time'] = []
    self.trip_sets[TS_route_index]['end_time'] = []
    self.trip_sets[TS_route_index]['delay'] = []
    self.trip_sets[TS_route_index]['total_distance'] = []
    self.trip_sets[TS_route_index]['total_time'] = []
    self.trip_sets[TS_route_index]['avg_speed'] = []
    self.trip_sets[TS_route_index]['avg_acceleration'] = []
    self.trip_sets[TS_route_index]['shape_id'] = []
    self.trip_sets[TS_route_index]['road_grade'] = []
    self.trip_sets[TS_route_index]['stop_density'] = []
    self.trip_sets[TS_route_index]['has_station'] = []
    self.trip_sets[TS_route_index]['dist_to_station'] = []
    self.trip_sets[TS_route_index]['time_at_station'] = []
    
  def get_trip_sets(self):
    return self.trip_sets

  def populate_trip_set(self, route, index, trip_data, delay):
    TS_route_index = 'TS_{}_{}'.format(route, index)
    if not self.trip_sets[TS_route_index]:
      raise Exception('Trip set does not exist')
    else:
      self.trip_sets[TS_route_index]['route_id'].append(trip_data['route_id'].iloc[0])
      self.trip_sets[TS_route_index]['trip_id'].append(trip_data['trip_id'].iloc[0])
      self.trip_sets[TS_route_index]['trip_headsign'].append(trip_data['trip_headsign'].iloc[0])
      self.trip_sets[TS_route_index]['origin'].append(trip_data['origin'].iloc[0])
      self.trip_sets[TS_route_index]['destination'].append(trip_data['destination'].iloc[0])
      self.trip_sets[TS_route_index]['start_time'].append(trip_data['start_time'].iloc[0])
      self.trip_sets[TS_route_index]['end_time'].append(trip_data['end_time'].iloc[0])
      self.trip_sets[TS_route_index]['delay'].append('{} min'.format(delay))
      self.trip_sets[TS_route_index]['total_distance'].append(trip_data['total_distance'].iloc[0])
      self.trip_sets[TS_route_index]['total_time'].append(trip_data['total_time'].iloc[0])
      self.trip_sets[TS_route_index]['avg_speed'].append(trip_data['avg_speed'].iloc[0])
      self.trip_sets[TS_route_index]['avg_acceleration'].append(trip_data['avg_acceleration'].iloc[0])
      self.trip_sets[TS_route_index]['shape_id'].append(trip_data['shape_id'].iloc[0]) # shape id for retreiving road profile
      self.trip_sets[TS_route_index]['road_grade'].append(trip_data['road_grade'].iloc[0])
      self.trip_sets[TS_route_index]['stop_density'].append(trip_data['stop_density'].iloc[0])
      self.trip_sets[TS_route_index]['has_station'].append(trip_data['has_station'].iloc[0])
      self.trip_sets[TS_route_index]['dist_to_station'].append(trip_data['dist_to_station'].iloc[0])
      self.trip_sets[TS_route_index]['time_at_station'].append(trip_data['time_at_station'].iloc[0])
  
  def update_delay(self, route, index, new_delay):
    TS_route_index = 'TS_{}_{}'.format(route, index)
    self.trip_sets[TS_route_index]['delay'] = '{} min'.format(new_delay)

In [ ]:
class trip_set_WC:
  def __init__(self):
      self.trip_sets_wc = {}

  def export_trip_sets(self, season):
    trip_set_path_wc = extracts + 'energy_consumption_data'
    winter = trip_set_path_wc + '/winter'
    summer = trip_set_path_wc + '/summer'
    spring = trip_set_path_wc + '/spring'
    fall = trip_set_path_wc + '/fall'
    if not os.path.exists(trip_set_path_wc):
      os.mkdir(trip_set_path_wc)
    if not os.path.exists(winter):
      os.mkdir(winter)
    if not os.path.exists(summer):
      os.mkdir(summer)
    if not os.path.exists(spring):
      os.mkdir(spring)
    if not os.path.exists(fall):
      os.mkdir(fall)
    if season == 'winter':
      for key in list(self.trip_sets_wc.keys()):
        route_trips = pd.DataFrame.from_dict(self.trip_sets_wc[key], orient='index').transpose()
        route_trips.to_csv('{}.csv'.format(key), index = False)
        shutil.move(local_path + '{}.csv'.format(key), winter + '/{}.csv'.format(key))
    elif season == 'summer':
      for key in list(self.trip_sets_wc.keys()):
        route_trips = pd.DataFrame.from_dict(self.trip_sets_wc[key], orient='index').transpose()
        route_trips.to_csv('{}.csv'.format(key), index = False)
        shutil.move(local_path + '{}.csv'.format(key), summer + '/{}.csv'.format(key))
    if season == 'spring':
      for key in list(self.trip_sets_wc.keys()):
        route_trips = pd.DataFrame.from_dict(self.trip_sets_wc[key], orient='index').transpose()
        route_trips.to_csv('{}.csv'.format(key), index = False)
        shutil.move(local_path + '{}.csv'.format(key), spring + '/{}.csv'.format(key))
    if season == 'fall':
      for key in list(self.trip_sets_wc.keys()):
        route_trips = pd.DataFrame.from_dict(self.trip_sets_wc[key], orient='index').transpose()
        route_trips.to_csv('{}.csv'.format(key), index = False)
        shutil.move(local_path + '{}.csv'.format(key), fall + '/{}.csv'.format(key))

  def create_trip_set(self, trip_identifier):
    self.trip_sets_wc[trip_identifier] = {}
    self.trip_sets_wc[trip_identifier]['route_id'] = []
    self.trip_sets_wc[trip_identifier]['trip_id'] = []
    self.trip_sets_wc[trip_identifier]['trip_headsign'] = []
    self.trip_sets_wc[trip_identifier]['origin'] = []
    self.trip_sets_wc[trip_identifier]['destination'] = []
    self.trip_sets_wc[trip_identifier]['start_time'] = []
    self.trip_sets_wc[trip_identifier]['end_time'] = []
    self.trip_sets_wc[trip_identifier]['energy_rate'] = []
    self.trip_sets_wc[trip_identifier]['energy_consumed'] = []
    self.trip_sets_wc[trip_identifier]['initial_soc'] = []
    self.trip_sets_wc[trip_identifier]['soc_before_charge'] = []
    self.trip_sets_wc[trip_identifier]['final_soc'] = []
    self.trip_sets_wc[trip_identifier]['total_time'] = []
    self.trip_sets_wc[trip_identifier]['total_distance'] = []
    self.trip_sets_wc[trip_identifier]['trip_type'] = []
    
  def get_trip_sets(self):
    return self.trip_sets_wc

  def populate_trip_set(self, trip_identifier, trip_data):
    if not self.trip_sets_wc[trip_identifier]:
      raise Exception('Trip set does not exist')
    else:
      self.trip_sets_wc[trip_identifier]['route_id'].append(trip_data['route_id'].iloc[0])
      self.trip_sets_wc[trip_identifier]['trip_id'].append(trip_data['trip_id'].iloc[0])
      self.trip_sets_wc[trip_identifier]['trip_headsign'].append(trip_data['trip_headsign'].iloc[0])
      self.trip_sets_wc[trip_identifier]['origin'].append(trip_data['origin'].iloc[0])
      self.trip_sets_wc[trip_identifier]['destination'].append(trip_data['destination'].iloc[0])
      self.trip_sets_wc[trip_identifier]['start_time'].append(trip_data['start_time'].iloc[0])
      self.trip_sets_wc[trip_identifier]['end_time'].append(trip_data['end_time'].iloc[0])
      self.trip_sets_wc[trip_identifier]['energy_rate'].append(trip_data['energy_rate'].iloc[0])
      self.trip_sets_wc[trip_identifier]['energy_consumed'].append(trip_data['energy_consumed'].iloc[0])
      self.trip_sets_wc[trip_identifier]['initial_soc'].append(trip_data['initial_soc'].iloc[0])
      self.trip_sets_wc[trip_identifier]['soc_before_charge'].append(trip_data['soc_before_charge'].iloc[0])
      self.trip_sets_wc[trip_identifier]['final_soc'].append(trip_data['final_soc'].iloc[0])
      self.trip_sets_wc[trip_identifier]['total_time'].append(trip_data['total_time'].iloc[0])
      self.trip_sets_wc[trip_identifier]['total_distance'].append(trip_data['total_distance'].iloc[0])
      self.trip_sets_wc[trip_identifier]['trip_type'].append(trip_data['trip_type'].iloc[0])
  
  def update_delay(self, trip_identifier, new_delay):
    self.trip_sets_wc[trip_identifier]['delay'] = '{} min'.format(new_delay)

In [ ]:
class EV_BUS:
  def __init__(self):
    self.ev_bus = {}

  def export_buses(self):
    bus_models_path = dataset_path + 'bus_models'
    if not os.path.exists(bus_models_path):
      os.mkdir(bus_models_path)
    bus_models = pd.DataFrame.from_dict(self.ev_bus, orient='index').transpose()
    bus_models.to_csv('ev_bus_models.csv', index = False)
    shutil.move(local_path + 'ev_bus_models.csv', bus_models_path + '/' + 'ev_bus_models.csv')

  def add_ev_bus(self, name, passenger_loading, batt_capacity):
    index = 0
    if len(list(self.ev_bus.keys())) != 0:
      index = len(list(self.ev_bus.keys()))
    BUS_IDENTIFIER = 'BEB_{}'.format(index) 
    self.ev_bus[BUS_IDENTIFIER] = {}
    self.ev_bus[BUS_IDENTIFIER]['Index'] = index
    self.ev_bus[BUS_IDENTIFIER]['Name'] = '{}'.format(name)
    self.ev_bus[BUS_IDENTIFIER]['Passenger'] = passenger_loading
    self.ev_bus[BUS_IDENTIFIER]['Capacity'] = batt_capacity
    self.ev_bus[BUS_IDENTIFIER]['SOC(%)'] = 100
    return index
  
  def update_soc(self, index, soc):
    BUS_IDENTIFIER = 'BEB_{}'.format(index)
    if not self.ev_bus[BUS_IDENTIFIER]:
      raise Exception('Bus does not exist')
    else:
      self.ev_bus[BUS_IDENTIFIER]['SOC(%)'] = soc

  def get_soc(self, index):
    BUS_IDENTIFIER = 'BEB_{}'.format(index)
    if not self.ev_bus[BUS_IDENTIFIER]:
      raise Exception('Bus does not exist')
    return self.ev_bus[BUS_IDENTIFIER]['SOC(%)']

  def get_ev_buses(self):
    return self.ev_bus

In [ ]:
class EnergyEstimator:
  def __init__(self):
    self.d_agg = 1 # driver aggressiveness in levels (3 levels) Editable
    self.rc = 1 # road condition in levels (3 levels) Editable
    self.p_den = 2 # people density in bus in levels (4 levels) Editable
    self.hvac = 3.7 # energy consumed due to aux in kW
    self.temp_spring = 2.3
    self.temp_summer = 16.7
    self.temp_fall = 5
    self.temp_winter = -11.7
    self.const = -0.782 # bias
    self.w_rg = 0.380 # weight for road grade
    self.w_soc = 0.0124 # weight for soc
    self.w_rc = 0.260 # weight for road condition
    self.w_hvac = 0.036 # weight for hvac
    self.w_pl = 0.005 # weight for passenger loader
    self.w_dagg = 0.065 # weight for driver agressiveness
    self.w_sd = 0.128 # weight for stop density
    self.w_spd = 0.007 # weight for average speed
  
  def set_aggressiveness(self, d_agg):
    self.d_agg = d_agg
  def set_road_condition(self, rc):
    self.rc = rc
  def set_passenger_density(self, p_den):
    self.p_den = p_den
  
  def calculate_energy_rate(self, road_grade, total_time, SOC, passenger_capacity,\
                            stop_density, avg_speed, total_distance,
                            battery_capacity, temp = 'summer', road_condition = 1,\
                            aggressiveness = 1, passenger_density = 2):
    # we assume that the auxillary is on for 60% of the trip (scaling factor of 0.6)
    # we assume driver aggressiveness is slow (level 1)
    # we assume road condition is dry (level 1)
    # we also assume a busy(ness) of level 2 (scaling factor of 1/2 of total passenger capacity)
    self.set_aggressiveness(aggressiveness)
    self.set_road_condition(road_condition)
    self.set_passenger_density(passenger_density)
    t = 0
    if temp == 'winter':
      t = self.temp_winter
    elif temp == 'summer':
      t = self.temp_summer
    elif temp == 'fall':
      t = self.temp_fall
    elif temp == 'spring':
      t = self.temp_spring
    duration = total_time/60 # convert from min to hr
    HVAC = (abs(20 - t) * 0.6 * duration) + self.hvac # kW
    passenger_loading = passenger_capacity * (1/self.p_den)
    energy_rate = self.const + (self.w_rg * road_grade) + (self.w_soc * SOC) +\
                 (self.w_rc * self.rc) + (self.w_hvac * HVAC) +\
                 (self.w_pl * passenger_loading) + (self.w_dagg * self.d_agg) +\
                 (self.w_sd * stop_density) + (self.w_spd * avg_speed) # kWh/km
    energy_consumed = energy_rate * total_distance # kWh
    final_soc = SOC - ((energy_consumed/battery_capacity)*100) # %
    if final_soc < 0: # chances are that estimated energy consumed would be more than is available
      final_soc = 0 
    return energy_rate, energy_consumed, final_soc
  
  def calculate_energy_charged(charger_size, time_taken, soc, battery_capacity):
    energy_charged = charger_size * (time_taken/60) # kWh
    final_soc = soc + ((energy_charged/battery_capacity)*100) # %
    return energy_charged, final_soc
  
  def get_required_recharge_time(battery_capacity, from_soc, to_soc, charger_type):
    if charger_type == 'swap':
      return 15 # 15 minutes to swap 
    elif charger_type == 'fast':
      charger_size = 120
    elif charger_type == 'slow':
      charger_size = 80
    time_required = (60*battery_capacity*((to_soc - from_soc)*0.01))/(charger_size)
    return time_required

In [ ]:
class charger:
  def __init__(self):
    self.ctype = {}
  
  def add_charger(self, location, c_type, rate_of_charge, operation_delay = 0):
    index = 0
    if not self.ctype:
      self.ctype['Index'] = ['chrg_{}_{}'.format(location[:1], index)]
      self.ctype['Location'] = [location]
      self.ctype['Type'] = [c_type]
      self.ctype['ROC'] = [rate_of_charge]
      self.ctype['OP_delay'] = [operation_delay]
    else:
      index = len(self.ctype['Index'])
      self.ctype['Index'].append('chrg_{}_{}'.format(location[:1], index))
      self.ctype['Location'].append(location)
      self.ctype['Type'].append(c_type)
      self.ctype['ROC'].append(rate_of_charge)
      self.ctype['OP_delay'].append(operation_delay)
  
  def update_charger(self, index, op_types = [], values = []):
    if not type(op_types) == list or not type(values) == list:
      raise Exception('expected a list of operation(s) and value(s)')
    elif index not in self.ctype['Index']:
      raise Exception('Invalid index. Reference list: {}'.format(list(self.ctype['Index'])))
    else:
      try:
        for op, value in zip(op_types, values):
          if op == 'Index' or op == 'Location':
            raise Exception('cannot alter index or location')
          else:
            self.ctype[op][int(index.split('_')[2])] = value
      except:
        raise Exception('Check that operation type matches: {}'.format(list(self.ctype.keys())))

  def get_charger(self):
    return self.ctype

In [ ]:
class stations:
  def __init__(self):
    self.charging_stations = {}

  def get_stations(self): # return list of station names
    return list(self.charging_stations.keys())
  
  def get_all_station_details(self): # return details of all stations
    return self.charging_stations

  def get_station_details(self, station): # return details of specified station
    return self.charging_stations[station]
  
  def add_station(self, stop_name): # add station
    self.charging_stations[stop_name] = {}

  def add_to_station(self, stop_name, c_type, rate_of_charge, operation_delay = 0): # add charger to station
    if stop_name not in list(self.charging_stations.keys()):
      raise Exception('station {} does not exist'.format(stop_name))
      print('Charging Stations: {}'.format(list(self.charging_stations.keys())))
    else:
      chg = charger()
      chg.add_charger(stop_name, c_type, rate_of_charge, operation_delay)
      chrg = chg.get_charger()
      if not self.charging_stations[stop_name]:
        self.charging_stations[stop_name]['charger_id'] = [chrg['Index'][-1]]
        self.charging_stations[stop_name]['type'] = [chrg['Type'][-1]]
        self.charging_stations[stop_name]['roc'] = [chrg['ROC'][-1]]
        self.charging_stations[stop_name]['op_delay'] = [chrg['OP_delay'][-1]]
        self.charging_stations[stop_name]['status'] = [1] # unoccupied(1) : occupied(0)
        self.charging_stations[stop_name]['Bus Queue'] = []
      else:
        self.charging_stations[stop_name]['charger_id'].append(chrg['Index'][-1])
        self.charging_stations[stop_name]['type'].append(chrg['Type'][-1])
        self.charging_stations[stop_name]['roc'].append(chrg['ROC'][-1])
        self.charging_stations[stop_name]['op_delay'].append(chrg['OP_delay'][-1])
        self.charging_stations[stop_name]['status'].append(1) # available(1) : unavailable(0)
        self.charging_stations[stop_name]['bus_queue'] = {}
        self.charging_stations[stop_name]['bus_queue'][chrg['Index'][-1]] = [] # in list: add (bus_index, time_to_charge)

  def get_wait_time(self, station): 
    sum = 0
    wait_times = {}
    for key, value in self.charging_stations[station]['bus_queue'].items():
      for i in value:
        sum += value[i][1]
      wait_times[key] = sum
    return wait_times # returns {charger_indexs : total_wait_times}

  def get_min_wait_times(self, cost):
    min_wait_time = {}
    for key, _ in self.charging_stations.keys():
      min_time = self.get_wait_time(key)
      min_time = list({k:v for k,v in min_time.items() if v == min(min_time.values())}.items())[0]
      min_time[1] = min_time[1] + cost
      min_wait_time[key] = min_time
    return min_wait_time # return a dictionary of station: min [charger_index, min_wait_time]

  def join_bus_queue(self, station, charge_index, bus_index, required_time):
    if charge_index not in list(self.charging_stations[station]['bus_queue'].keys()):
      raise Exception('index does not exist')
    else:
      self.charging_stations[station]['bus_queue'][charge_index].append([bus_index, required_time])
      self.update_charger_availability(station, charge_index)
  
  def update_charger_availability(self, station, charge_index):
    index = [idx for idx, key in \
               enumerate(list(self.charging_stations[station]['charger_id']\
                              .items())) if key[0] == charge_index]
    if len(self.charging_stations[station]['bus_queue'][charge_index]) == 0:
      self.charging_stations[station]['status'][index] = 1
    else:
      self.charging_stations[station]['status'][index] = 0
    return self.charging_stations[station]['status'][index]

  def done_charging(self, station, charge_index):
    self.charging_stations[station]['bus_queue'][charge_index].pop(0)
    self.update_charger_availability(station, charge_index)

In [ ]:
# helper function to convert time string to total minutes for number comparison
def tomins(time):
  x = t.strptime(time, FMT)
  return (datetime.timedelta(hours=x.tm_hour,\
                            minutes=x.tm_min,\
                            seconds=x.tm_sec).total_seconds())/60

def get_total_time(start_time, end_time):
  return tomins(end_time) - tomins(start_time)

In [ ]:
# trip assignment (without charging)
tripsets = trip_set()
directory = extracts + 'Jan/20220124_route_data/'
for filename in os.listdir(directory):
  current_file = os.path.join(directory, filename)
  route = filename.split('.')[0] # getting the route_id
  temp_df = pd.read_csv(current_file) # reading the current route data file into a df
  unassigned = []
  assigned = []
  index = 0
  for trip_id in temp_df['trip_id']:
    unassigned.append(trip_id)
  while unassigned:
    current_trip = unassigned.pop(0)
    tripsets.create_trip_set(route, index)
    ctd = temp_df.loc[temp_df['trip_id'] == current_trip] # current trip data
    tripsets.populate_trip_set(route, index, ctd, 0)
    curr_dest = ctd['destination'].iloc[0]
    curr_end = ctd['end_time'].iloc[0]
    for trip in unassigned:
      curr_trip = temp_df.loc[temp_df['trip_id'] == trip]
      next_origin = curr_trip['origin'].iloc[0]
      next_start = curr_trip['start_time'].iloc[0]
      # if current trip meets given criteria, add to trip set and make it the leading trip
      if next_origin == curr_dest and tomins(next_start) >= tomins(curr_end):
        tripsets.populate_trip_set(route, index, curr_trip, get_total_time(curr_end, next_start))
        assigned.append(curr_trip['trip_id'].iloc[0]) # add to assigned list
        curr_dest = curr_trip['destination'].iloc[0]
        curr_end = curr_trip['end_time'].iloc[0]
    index += 1
    unassigned = list(filter(lambda trip: trip not in assigned, unassigned))

In [ ]:
tripsets.export_trip_sets()

In [ ]:
def add_to_time(time, mins):
  x = t.strptime(time, FMT)
  final_time = x + datetime.timedelta(minutes = mins)
  return final_time.strftime(FMT)

In [ ]:
energyCalculator = EnergyEstimator()
directory = extracts + 'trip_sets/'
seasons = ['winter', 'summer', 'spring', 'fall']
for season in seasons:
  trip_sets =  trip_set_WC()
  for filename in os.listdir(directory):
    current_file = os.path.join(directory, filename)
    temp_df = pd.read_csv(current_file)
    trip_identifier = filename.split('.')[0]
    trip_sets.create_trip_set(trip_identifier)
    # initial settings
    initial_soc = 100
    passenger_capacity = 41 # number of available seatings
    battery_capacity = 80 # kWh
    for trip_id in temp_df['trip_id']:
      unassigned.append(trip_id)
    while unassigned:
      next_trip_data = {}
      curr_trip = unassigned.pop(0)
      curr_trip_data = temp_df.loc[temp_df['trip_id'] == curr_trip]
      curr_trip_energy = energyCalculator.calculate_energy_rate()
  trip_sets.export_trip_sets(season)